<a href="https://colab.research.google.com/github/kanru-wang/Distributed_Training_and_Quantization_Pruning_Distillation/blob/main/multi_gpu_mirrored_strategy_distributed_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-GPU Mirrored Strategy

Multi-GPU mirrored strategy distributed training for Fashion MNIST.

There's only 1 device that is available, which is enough for demonstrating the distribution strategies.

See: https://www.coursera.org/learn/custom-distributed-training-with-tensorflow/lecture/21zgD/multi-gpu-mirrored-strategy-code-walkthrough

In [1]:
import tensorflow as tf
import numpy as np
import os

## Setup Distribution Strategy

In [2]:
# Note that it generally has a minimum of 8 cores, but if the GPU has
# less, will need to set this. E.g., if one of the GPUs only has 4 cores
# os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = "4"

# If the list of devices is not specified in the
# `tf.distribute.MirroredStrategy` constructor, it will be auto-detected.
# If you have *different* GPUs, will probably have to set up cross_device_ops like this
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


## Prepare the Data

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Adding a dimension to the array
# Old train_images.shape == (60000, 28, 28)
# New train_images.shape == (60000, 28, 28, 1)
# Do so because the first layer in the model is a convolutional layer
# and it requires a 4D input (batch_size, height, width, channels).
# batch_size dimension will be added later on.
train_images = train_images[..., None]
test_images = test_images[..., None]

# Normalize to [0, 1]
train_images = train_images / np.float32(255)
test_images = test_images / np.float32(255)

BUFFER_SIZE = len(train_images)
BATCH_SIZE_PER_REPLICA = 64
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(BUFFER_SIZE).batch(GLOBAL_BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(GLOBAL_BATCH_SIZE)

# Create Distributed Datasets from the datasets
train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
test_dist_dataset = strategy.experimental_distribute_dataset(test_dataset)

## Define the Model

In [4]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])
    return model

## Configure custom training

Instead of `model.compile()`, will do custom training within a strategy scope.

If there are two GPUs, the printed output of `per_example_loss` should look like this:

    Tensor("sparse_categorical_crossentropy/weighted_loss/Mul:0", shape=(48,), dtype=float32, device=/job:localhost/replica:0/task:0/device:GPU:0)
    Tensor("replica_1/sparse_categorical_crossentropy/weighted_loss/Mul:0", shape=(48,), dtype=float32, device=/job:localhost/replica:0/task:0/device:GPU:1)
    
Note in particular that `replica_0` isn't named in the `weighted_loss` -- the first is unnamed, the second is `replica_1` etc

In [5]:
with strategy.scope():
    # Use sparse categorical crossentropy, but instead of having the loss function
    # manage the map reduce (which is how the losses get aggregated) across GPUs for us,
    # will do it ourselves with a simple algorithm.
    # Set reduction to `none` so we can do the reduction afterwards and divide by global batch size.
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True,
        reduction=tf.keras.losses.Reduction.NONE
    )

    def compute_loss(labels, predictions):
        # Notice that per_example_loss will have an entry per GPU,
        # i.e. the loss for each replica
        per_example_loss = loss_object(labels, predictions)
        print(per_example_loss)
        # tf.print(per_example_loss)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size=GLOBAL_BATCH_SIZE)

    # Reduce by averaging the losses
    test_loss = tf.keras.metrics.Mean(name='test_loss')

    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

    optimizer = tf.keras.optimizers.Adam()

    # Create the model within the scope
    model = create_model()

## Train and Test Steps Functions

In [6]:
# `run` replicates the provided computation and runs it with the distributed input.
@tf.function
def distributed_train_step(dataset_inputs):
    per_replica_losses = strategy.run(train_step, args=(dataset_inputs,))
    # tf.print(per_replica_losses.values)
    # print(per_replica_losses)
    return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)

def train_step(inputs):
    images, labels = inputs
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = compute_loss(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_accuracy.update_state(labels, predictions)
    return loss

#######################
# Test Steps Functions
#######################
@tf.function
def distributed_test_step(dataset_inputs):
    return strategy.run(test_step, args=(dataset_inputs,))

def test_step(inputs):
    images, labels = inputs

    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss.update_state(t_loss)
    test_accuracy.update_state(labels, predictions)

## Training Loop

We can now start training the model.

In [7]:
EPOCHS = 10
for epoch in range(EPOCHS):
    # Do Training
    total_loss = 0.0
    num_batches = 0
    for batch in train_dist_dataset:
        total_loss += distributed_train_step(batch)
        num_batches += 1
    train_loss = total_loss / num_batches

    # Do Testing
    for batch in test_dist_dataset:
        distributed_test_step(batch)

    template = ("Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, " "Test Accuracy: {}")

    print(template.format(epoch+1, train_loss, train_accuracy.result()*100, test_loss.result(), test_accuracy.result()*100))

    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

Tensor("sparse_categorical_crossentropy/weighted_loss/Mul:0", shape=(64,), dtype=float32, device=/job:localhost/replica:0/task:0/device:GPU:0)
Tensor("sparse_categorical_crossentropy/weighted_loss/Mul:0", shape=(64,), dtype=float32, device=/job:localhost/replica:0/task:0/device:GPU:0)
Tensor("sparse_categorical_crossentropy/weighted_loss/Mul:0", shape=(32,), dtype=float32, device=/job:localhost/replica:0/task:0/device:GPU:0)
Epoch 1, Loss: 0.5183785557746887, Accuracy: 81.40666198730469, Test Loss: 0.3894541561603546, Test Accuracy: 86.29000091552734
Epoch 2, Loss: 0.33887457847595215, Accuracy: 87.81832885742188, Test Loss: 0.3472968637943268, Test Accuracy: 87.95999908447266
Epoch 3, Loss: 0.2911539673805237, Accuracy: 89.37332916259766, Test Loss: 0.31044477224349976, Test Accuracy: 88.73999786376953
Epoch 4, Loss: 0.2592061758041382, Accuracy: 90.58000183105469, Test Loss: 0.29861727356910706, Test Accuracy: 89.18000030517578
Epoch 5, Loss: 0.2356150597333908, Accuracy: 91.37667083